# Lab #3 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/basic-operations-workshop/blob/main/lab3.ipynb)
1. Install dependencies
2. Create a pinecone index 
3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset
4. Create a local parquet backup of your image embeddings
5. Insert the fashion-mnist embeddings into Pinecone
6. Run a nearest neighbor search on a sample image that is not in the training dataset
7. Run a nearest neighbor search on 100 random test images that are not in the training dataset
8. Run a load test script to simulate 10 concurrent users querying the index
9. TEARDOWN: Delete the index 

# 1. Install Pinecone client 
Use the following shell command to install Pinecone:

In [ ]:
!pip install -U "pinecone-client[grpc]" "python-dotenv" "torch" "torchvision" "pillow" "ftfy" "regex" "tqdm" "git+https://github.com/openai/clip.git" "datasets" "locust"

# 2. Create a pinecone index 
We will create an index that will be used to load/query a hugging face dataset.

In [1]:
from dotenv import load_dotenv
import os
import pinecone

load_dotenv('.env')

PINECONE_INDEX_NAME = os.environ['PINECONE_INDEX_NAME']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_ENVIRONMENT = os.environ['PINECONE_ENVIRONMENT']
METRIC = os.environ['METRIC']
DIMENSIONS = int(os.environ['DIMENSIONS'])
INDEX_NAMESPACE = os.environ['INDEX_NAMESPACE']

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone.create_index(PINECONE_INDEX_NAME, dimension=DIMENSIONS, metric=METRIC, pods=1, replicas=1, pod_type="s1")
pinecone.describe_index(PINECONE_INDEX_NAME)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


IndexDescription(name='james-williams', metric='cosine', replicas=1, dimension=512.0, shards=1, pods=1, pod_type='s1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

# 3. Load public image dataset(fashion-mnist) and create vector embeddings from the dataset

Use the following shell command to download the [fashion-mnist](https://huggingface.co/datasets/fashion_mnist) training dataset from Hugging Face so that we can create vector embeddings that uses a label(image class) as meta-data from this dataset. The meta-data labels mappings are:

| Label  | Description |
| ------ | ----------- |
| 0      | T-shirt/top |
| 1      | Trouser     |
| 2      | Pullover    |
| 3      | Dress       |
| 4      | Coat        |
| 5      | Sandal      |
| 6      | Shirt       |
| 7      | Sneaker     |
| 8      | Bag         |
| 9      | Ankle boot  |

The Fashion-MNIST dataset is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

The accuracy you can achieve depends on the model and the preprocessing steps you use. Here's a rough guideline for what you might expect with some classic machine learning algorithms:

1. **Random Forest:** Around 85-89% accuracy.
2. **Support Vector Machines (SVM):** Around 85-90% accuracy, depending on kernel and hyperparameters.
3. **k-Nearest Neighbors (k-NN):** Around 85-88% accuracy.
4. **Logistic Regression:** Around 82-85% accuracy.
5. **Gradient Boosting Machines (e.g., XGBoost):** Around 87-90% accuracy.

Keep in mind these numbers are approximate and can vary based on the exact preprocessing, feature extraction, and hyperparameter tuning you do. In general, deep learning models, especially Convolutional Neural Networks (CNNs), tend to perform better on image classification tasks like Fashion-MNIST, potentially reaching over 90-95% accuracy.

But for classic machine learning models, anything in the 85-90% range can be considered a reasonable result for the Fashion-MNIST dataset. It reflects a model that has learned something meaningful from the data but isn't necessarily state-of-the-art for this particular task.

In [2]:
from datasets import load_dataset
from tqdm.auto import tqdm  # progress bar
from PIL import Image
import torch
import clip
import time
import numpy as np

#  Load the fashion-mnist dataset - only retrieve 6K random images (10% of total dataset)
dataset = load_dataset("fashion_mnist")['train'].shuffle(seed=42).select(range(0,6000))
#dataset = load_dataset("fashion_mnist")['train']

label_descriptions = {0: "T-shirt/top", 
           1: "Trouser",
           2: "Pullover",
           3: "Dress",
           4: "Coat",
           5: "Sandal",
           6: "Shirt",
           7: "Sneaker",
           8: "Bag",
           9: "Ankle boot"}

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)
   
# Generate vector embeddings for each image in the dataset
id = 0
vectors = []
for img in tqdm(dataset, total=dataset.num_rows, desc='Images', position=0):
    with torch.no_grad():
        id += 1
        image_pp = preprocess(img['image']).unsqueeze(0).to(device)
        image_features = model.encode_image(image_pp)
        embedding = image_features.cpu().numpy().squeeze().tolist()
        meta_data = {"description": label_descriptions[img["label"]], "timestamp": time.time()}
        vectors.append({'id': str(id),
                        'values': embedding,
                        'metadata': meta_data})

Images: 100%|██████████| 6000/6000 [03:11<00:00, 31.33it/s]


# 4. Create a local parquet backup of your image embeddings

This is good practice because generating embeddings can be expensive and time consuming when calling hosted models like OpenAI. As you can see, even locally generated embeddings are time consuming.

In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df = pd.DataFrame(vectors)
df.to_parquet('fashion-mnist-clip.parquet')

# 5. Insert the fashion-mnist embeddings into Pinecone

The best way to do bulk updates is by batching the dataset. We will also use a namespace for the data. 

In [4]:
from tqdm.auto import tqdm  # progress bar
import pinecone
import itertools

# Read Parquet file into a DataFrame
df = pd.read_parquet('fashion-mnist-clip.parquet')
df['values'] = df['values'].apply(lambda x: x.tolist())

# Convert DataFrame to a list of dictionaries
data_list = df.to_dict(orient='records')

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

index = pinecone.Index(PINECONE_INDEX_NAME)

# Obtain the upsert embeddings in batches of 100
batch_size = 100
id = 0
for vector_batch in tqdm(chunks(data_list, batch_size=batch_size), total=(len(vectors) / batch_size)):
   index.upsert(vector_batch, namespace=INDEX_NAMESPACE)

100%|██████████| 60/60.0 [00:34<00:00,  1.72it/s]


# 6. Run a nearest neighbor search on a sample image that is not in the training dataset

Download a sneaker image file from github that we will use to run a query to see if pinecone search returns the correct description "Sneaker". A query returns the correct result if the most common top_k description matches the test image.

You can change the top_k from 10 to 1 to 20 to see if the ANN results vary.

In [5]:
import pinecone
from PIL import Image
import torch
import clip
import requests

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) 

def image_to_embedding():
    
    url = "https://github.com/pinecone-io/basic-operations-workshop/blob/main/sneaker.jpeg?raw=true"
    response = requests.get(url)
    with open("sneaker.jpeg", "wb") as file:
      file.write(response.content)
    image_pp = preprocess(Image.open("./sneaker.jpeg")).unsqueeze(0).to(device)
    with torch.no_grad():
      embedding = model.encode_image(image_pp).squeeze().tolist()
    
    return embedding

index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10

query_result = index.query(
  vector = image_to_embedding(),
  namespace=INDEX_NAMESPACE,
  top_k=top_k,
  include_values=False,
  include_metadata=True
)

top_k_success = False
match_cnt = 0
miss_categories = set()

my_list = query_result.matches
descriptions = [entry['metadata']['description'] for entry in my_list]
most_common_item = max(set(descriptions), key=descriptions.count)

if most_common_item == "Sneaker":
      top_k_success = True

for match in query_result.matches:
  if match.metadata['description'] == "Sneaker":
    match_cnt += 1
    top_k_contains = True
  else:
    miss_categories.add(match.metadata['description'])

print(f"Most common item matching result: {top_k_success}")
print(f"top_k: {top_k} match percentage is: {match_cnt/top_k * 100}%")
print(f"Match miss categories: {miss_categories} exepected 'Sneaker'")

Most common item matching result: True
top_k: 10 match percentage is: 70.0%
Match miss categories: {'Ankle boot'} exepected 'Sneaker'


# 7. Run a nearest neighbor search on 100 random test images that are not in the training dataset

Select 100 random test images. Keep in mind the model was NOT trained against these images. Obtain the percentage of pinecone queries that return the correct result in top_k. Feel free to play around with the "top_k" setting to see if you can increase the hit percentage. 

A query returns the correct result if the most common top_k description matches the test image.

In [10]:
import clip
import torch
import pinecone
from datasets import load_dataset
from tqdm.auto import tqdm 
from collections import Counter

test_dataset = load_dataset("fashion_mnist")['test'].shuffle().select(range(0, 100))
#test_dataset = load_dataset("fashion_mnist")['test']

# Check to see if GPU is aviailable
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device) 

# Generate vector embeddings for each image in the dataset
test_vectors = []
for img in tqdm(test_dataset, total=test_dataset.num_rows):
  image_pp = preprocess(img['image']).unsqueeze(0).to(device)
  embedding = model.encode_image(image_pp).squeeze().tolist()
    
  test_vectors.append({'embedding': embedding,
                        'description': label_descriptions[img["label"]]})
    
index = pinecone.Index(PINECONE_INDEX_NAME)
top_k = 10
top_k_success_cnt = 0

for v in test_vectors:

  top_k_success = False

  query_result = index.query(
    vector = v['embedding'],
    namespace=INDEX_NAMESPACE,
    top_k=top_k,
    include_values=False,
    include_metadata=True
  )

  my_list = query_result.matches
  descriptions = [entry['metadata']['description'] for entry in my_list]
  most_common_item = max(set(descriptions), key=descriptions.count)

  if most_common_item == v['description']:
      top_k_success = True
  
  if top_k_success:
    top_k_success_cnt += 1

print(f"top_k success rate: {top_k_success_cnt / (len(test_vectors)) * 100}%")

100%|██████████| 100/100 [00:04<00:00, 22.98it/s]


top_k success rate: 86.0%


# 8. Run a load test script to simulate 10 concurrent users querying the index

Locust.io is an open-source load testing tool written in Python. It allows you to define user behaviour with Python code and simulate millions of simultaneous users to bombard a system with traffic to test its resilience under heavy load. The (locustfile.py)[./locustfile.py] script re-uses the logic in step #6 to query pinecone. It has a custom event hook that denotes a failure if the top_k result set does not match the search image description. This script will likely fail with a low error rate but you can increase top_k to get a 100% pass rate. The locust summary includes P50 to P100 response time percentiles and QPS(req/s).

In [11]:
%%bash
locust -f locustfile.py --headless -u 10 -r 1 --run-time 60s --host https://pinecone.io --only-summary

Generated embeddings for 100 images


[2023-08-04 17:44:32,907] williamsj-MacBook-Pro/INFO/locust.main: Run time limit set to 60 seconds
[2023-08-04 17:44:32,907] williamsj-MacBook-Pro/INFO/locust.main: Starting Locust 2.16.0
[2023-08-04 17:44:32,908] williamsj-MacBook-Pro/INFO/locust.runners: Ramping to 10 users at a rate of 1.00 per second
[2023-08-04 17:44:41,939] williamsj-MacBook-Pro/INFO/locust.runners: All users spawned: {"PineconeUser": 10} (10 total users)
[2023-08-04 17:45:32,909] williamsj-MacBook-Pro/INFO/locust.main: --run-time limit reached, shutting down
[2023-08-04 17:45:32,913] williamsj-MacBook-Pro/INFO/locust.main: Shutting down (exit code 0)
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
grpc     query_pinecone                       

# 9. TEARDOWN: Delete the index 
# WARNING: This next step will delete the PINECONE_INDEX_NAME index and all data in it. DO NOT RUN THIS UNTIL YOU ARE READY OR MANUALLY REMOVE THE INDEX INSTEAD!!! 

In [ ]:
if PINECONE_INDEX_NAME in pinecone.list_indexes():
    pinecone.delete_index(PINECONE_INDEX_NAME)
    
pinecone.list_indexes()